In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from gensim import corpora, models
from textblob import TextBlob
import operator
import mord

from src import *

# Feature Engineering

In [ ]:
# Read data
db_file = './data/db_master_clean.pickle'
df = pd.read_pickle(db_file)

#### prepare feature matrix "X" and target "y" for Machine Learning

In [ ]:
target_filter = '12'
#target_filter = '134'

X, y = ml_preprocessing(df, filter=target_filter)
print X.shape, y.shape

In [ ]:
def X_lsi_additional(X_lsi, y):

    print X_lsi.shape, y.shape
    mask = [len(list_elem) == num_topics for list_elem in X_lsi]
    y = y[mask]
    X_lsi = X_lsi[mask]
    print X_lsi.shape, y.shape

    _X_lsi = [elem for list_elem in X_lsi for elem in list_elem]
    _X2_lsi = [_X_lsi[i:i+num_topics] for i in range(0, len(_X_lsi), num_topics)]
    X_lsi = np.array(_X2_lsi)
    print X_lsi.shape, y.shape
    
    return X_lsi, y

    
def create_features_semantic(X):   
    
    #def print_feature_report(X_tfidf, min_df, max_df): 
    #    print ">>> feature report:"
    #    X_length, _ = Xtfidf[:,0].shape
    #    print "%d texts" % X_length
    #    print "required minimal word occurence of %.1f" % min_df
    #    print "required maximal word occurence of %.1f" % max_df
    #    print "global number of different words in texts:", X_tfidf.count_nonzero()
    
    # filter words
    #cv_dict = count_vectorizer.vocabulary_
    #no_samples, no_words = X_counts.todense().shape
    #x = {}
    #for key in cv_dict:
    #    x[key] = X_counts.todense().sum(axis=0)[0,cv_dict[key]]
    #sorted_x = sorted(x.items(), key=operator.itemgetter(1))
    #print '20 most frequently occurring words:'
    #filtered_words = [elem for elem, number in sorted_x[:-21:-1]]
    
    
    X_tok = [tokenizeText(cleanText(elem)) for elem in X]
    #X_tokk = [[tok for tok in tok_list if tok in filtered_words] for tok_list in X_tok]
    X_tokk = X_tok
    dictionary = corpora.Dictionary(X_tokk)
    print dictionary

    ###### Tfidf
    corpus = [dictionary.doc2bow(elem) for elem in X_tokk]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]

    ###### Latent Semantic Analysis
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)
    corpus_lsi = lsi[corpus_tfidf]
    print '\n>>> Lsi topics:'
    _lsi = [elem for elem in lsi.print_topics(num_topics)]
    for elem in _lsi:
        print elem
    
    print '\n>>> First few Lsi features:'
    for i, doc in enumerate(corpus_lsi): # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
        print(doc)
        if i == 5:
            break

    ###### Turn into feature matrix
    _X_lsi = [[topic_correlation for (_,topic_correlation) in doc] for doc in corpus_lsi]
    X_lsi = np.array(_X_lsi)#.reshape(len(lsi_features_clean)/num_topics,num_topics)
    
    #X_lsi, y = X_lsi_additional(X_lsi, y)
    
    return X_lsi

In [ ]:
print '\n--------------------------------'
print '--------------------------------\n'

if 'semantic' in feature_type:
    %time Xlsi = create_features_semantic(X)